<a href="https://colab.research.google.com/github/sampycool/Gradient_Descent_Algorithm_Using_Numpy/blob/master/SSES_data_read_offset_core_well_v25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import glob
import chardet
import torch
import torch.nn as nn
import shutil

In [ ]:

def similarity(cr_wl, ofst_wl, core_well_nm, ofst_well_nm,depth_type):
  cnt_sim = 0
  w_size_sim = len(cr_wl)

  folder_path = '/content/drive/My Drive/SSES/sim_files/'+core_well_nm+'/'
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  plots_path = folder_path + ofst_well_nm +'/'
  if not os.path.exists(plots_path):
    os.makedirs(plots_path)


  list_dot_sim = []
  list_cosine_sim = []
  cnt = 0
  for x in range(0, len(ofst_wl), 1):

    ofst_wndw = ofst_wl[cnt_sim:cnt_sim+len(cr_wl)]
    if len(ofst_wndw) != len(cr_wl):
      break
    else:

      cr_wl_t =   torch.from_numpy(cr_wl['GRD_SC'].values)
      ofst_wndw_t =   torch.from_numpy(ofst_wndw['GRD_SC'].values)

      dot_t = torch.dot(cr_wl_t,ofst_wndw_t)

      cosine = nn.CosineSimilarity(dim=0, eps=1e-3)
      output = cosine(cr_wl_t, ofst_wndw_t)

      list_dot_sim.append(str(dot_t.numpy()))
      list_cosine_sim.append(str(output.numpy()))

      # plot the graphs in sub plots
      # if depth_type == 'md':

      #   # print('in MD plot ' + ofst_well_nm.replace('alt_','') +str(cnt+1) )
      #   labels = [core_well_nm + '_window_' +str(cnt+1), ofst_well_nm.replace('alt_','')]
      #   fig, axs = plt.subplots(2)
      #   l1 = axs[0].plot(cr_wl['MD'].values,cr_wl.iloc[:,2].values)
      #   l2 = axs[1].plot(ofst_wndw['MD'].values,ofst_wndw.iloc[:,1].values,'tab:green')
      #   fig.legend([l1, l2], labels=labels, loc="upper right")
      #   # fig.suptitle('Cosine Similarity : '+str(output.numpy()))
      #   fig.suptitle('Cosine Similarity : '+str(output.numpy())[:6], fontsize=8)
      #   plt.subplots_adjust(right=0.9)
      #   plt.savefig(plots_path + ofst_well_nm.replace('alt_','') + '_' + str(cnt+1) +'.png')
      # else:
      #   # print('in TVD plot ' + ofst_well_nm.replace('alt_','') +str(cnt+1) )
      #   labels = [core_well_nm + '_window_' +str(cnt+1), ofst_well_nm.replace('alt_','')]
      #   fig, axs = plt.subplots(2)
      #   l1 = axs[0].plot(cr_wl['TVD'].values,cr_wl.iloc[:,1].values)
      #   l2 = axs[1].plot(ofst_wndw['MD'].values,ofst_wndw.iloc[:,1].values,'tab:green')
      #   fig.legend([l1, l2], labels=labels, loc="upper right")
      #   fig.suptitle('Cosine Similarity : '+str(output.numpy())[:6], fontsize=8)
      #   # fig.suptitle('This is a somewhat long figure title', fontsize=16)
      #   plt.subplots_adjust(right=0.9)
      #   plt.savefig(plots_path + ofst_well_nm.replace('alt_','') + '_' + str(cnt+1) +'.png')
      cnt = cnt +1





    cnt_sim = cnt_sim + 1


  arr_dot_sim = np.array(list_dot_sim)
  arr_cos_sim = np.array(list_cosine_sim)

  df_sim = pd.DataFrame(np.array(list_dot_sim), columns=['dot'])
  df_sim = pd.concat([df_sim, pd.DataFrame(arr_cos_sim, columns=['cosine'])],axis =1)
  df_sim.to_excel(folder_path +'/'+ ofst_well_nm.replace('alt_','') +'.xlsx')






In [ ]:

path  = '/content/drive/My Drive/SSES/needed/excel_files/'
offset_path  = '/content/drive/My Drive/SSES/needed/'


fl_ptrn_1 = offset_path + '*alt*.csv'
print('>> ',glob.glob(fl_ptrn_1))               # returns a list of file names with a *alt*.csv
offset_well_names_alt = glob.glob(fl_ptrn_1)


fl_ptrn = path + '*survey_station*.xls*'
# returns a list of file names with a *_*.xlsx with their paths
core_well_names = glob.glob(fl_ptrn)          # returns 143 file names
lst_df =[]

for i in range(len(core_well_names)):


    lst_sheet = pd.ExcelFile(core_well_names[i]).sheet_names
    df = pd.read_excel(core_well_names[i],lst_sheet[-1])
    srs_core_grd = df['GRD']
    srs_core_tvd = df['TVD']      #float64
    srs_core_tvd = srs_core_tvd.astype(int)
    df['TVD'] = srs_core_tvd
    srs_core_grd_np = srs_core_grd.values.reshape(len(srs_core_grd), 1)
    srs_core_md = df['MD'].iloc[:-1]
    srs_core_tvd = df['TVD'].iloc[:-1]
    scaler = MinMaxScaler()
    model=scaler.fit(srs_core_grd_np)
    srs_core_grd_sc =model.transform(srs_core_grd_np)
    df = pd.concat([df, pd.DataFrame(srs_core_grd_sc, columns=['GRD_SC'])], axis=1)
    tvd_flag = False

    for j in range(len(offset_well_names_alt)):

        df_alt = pd.read_csv(offset_well_names_alt[j])
        df_alt_grd = df_alt[df_alt.columns[-1]]
        df_alt_grd_np = df_alt_grd.values.reshape(len(df_alt_grd), 1)
        scaler = MinMaxScaler()
        model = scaler.fit(df_alt_grd_np)
        df_alt_grd_sc = model.transform(df_alt_grd_np)
        df_alt = pd.concat([df_alt, pd.DataFrame(df_alt_grd_sc, columns=['GRD_SC'])], axis=1)


        df_alt_md = df_alt.iloc[:,0]            # 0 is the index of the 1st column i.e 'MD'
        w_size = len(srs_core_md)
        cnt = 0
        flag = False

        for ofst_window in df_alt_md.rolling(window= (w_size), step = 1):

          nw_ofst_window = pd.Series()
          if len(ofst_window)  != len(srs_core_md):
            continue
          else:
            cnt = cnt + 1
            srs_core_md = srs_core_md.reset_index(drop=True)
            ofst_window = ofst_window.reset_index(drop=True)
            xs = (srs_core_md  == ofst_window)
            true_cnt = xs.value_counts().filter(like='True').iloc[:]            # pandas Series

            if len(true_cnt) > 0:
              ow_strt_idx = df_alt_md[df_alt_md==ofst_window.iloc[0]].index.values[0]   # return the first element of that list

              if ow_strt_idx >= 50:
                nw_ofst_strt_idx =  ow_strt_idx - 50
                pre_ofst_window = df_alt_md[nw_ofst_strt_idx:ow_strt_idx]
                nw_ofst_window = pd.concat([pre_ofst_window, ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)

              else:
                pre_ofst_window = df_alt_md[:ow_strt_idx]                       #   all elements form the beginning
                nw_ofst_window = pd.concat([pre_ofst_window, ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)     # pandas series


              ow_end_idx = df_alt_md[df_alt_md==ofst_window.iloc[-1]].index.values[0]   # return the last element of that list; step (a)
              diff =  df_alt_md.index.stop - ow_end_idx

              if diff >=50:
                post_ofst_window = df_alt_md[ow_end_idx+1:ow_end_idx+1+50]
                nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)     # pandas series

              else:
                post_ofst_window = df_alt_md[ow_end_idx+1:]
                nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)     # pandas series


              c_well_md = srs_core_md.values
              o_well_md = nw_ofst_window.values
              c_well_grd = df['GRD_SC'].values
              o_well_grd = df_alt['GRD_SC'].values


              df_core_window = df[df['MD'].isin(c_well_md)][df.columns]
              df_ofst_window = df_alt[df_alt['MD'].isin(o_well_md)][df_alt.columns]

              # call similarity fn
              core_well_nm = core_well_names[i].split('.')[0].split('/')[-1]
              ofst_well_nm = offset_well_names_alt[j].split('.')[0].split('/')[-1]
              similarity(df_core_window, df_ofst_window, core_well_nm, ofst_well_nm,'md')


              flag = True
              tvd_flag = True
              break

        if flag == False:
          fst_val_ss = srs_core_md.iloc[0]  # first val in the MD OR TVD columns
          result = fst_val_ss in df_alt_md.values
          if result == True:

            strt_idx = df_alt_md[df_alt_md==fst_val_ss].index.values
            new_ofst_window = df_alt_md[strt_idx[0]:]       # pandas Series ..... this is the length of the offset well index
            # again 2 cases ...
            # case a) the last element of offset well is present in the core well
            # [this happens when length of offset well < length of core well ]..


            # case b) the last element of offset well is absent in the core well
            #[this happens when length of offset well > length of core well ]..

            if (new_ofst_window.iloc[-1] in srs_core_md.values):
              print('case a')
              last_ele = new_ofst_window.iloc[-1]       # check
              end_idx = srs_core_md[srs_core_md==last_ele].index.values[0]      # end_idx in the core well where we cut off the segment for matching
              srs_core_md_for_match = srs_core_md[:end_idx+1]

              # so we have to add 50 elements or lesser to the offset well over here to the newly formed series.

              if strt_idx[0] >= 50:     # check if the start index of the offset well > 50
                nw_ofst_strt_idx =  strt_idx[0] - 50
                pre_ofst_window = df_alt_md[nw_ofst_strt_idx:strt_idx[0]]
                nw_ofst_window = pd.concat([pre_ofst_window, new_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)
                # call similarity fn

              else:
                pre_ofst_window = df_alt_md[:strt_idx[0]]               #   all elements form the beginning
                nw_ofst_window = pd.concat([pre_ofst_window, new_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series

              # ADD 50 ELEMENTS AT THE END

              ow_end_idx = df_alt_md[df_alt_md==ofst_window.iloc[-1]].index.values[0]   # return the last element of that list; step (a)
              diff =  df_alt_md.index.stop - ow_end_idx

              if diff >=50:
                post_ofst_window = df_alt_md[ow_end_idx+1:ow_end_idx+1+50]
                nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series

              else:
                post_ofst_window = df_alt_md[ow_end_idx+1:]
                nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series

              tvd_flag = True

            else:
              print('case b')


            c_well_md = srs_core_md.values
            o_well_md = nw_ofst_window.values
            c_well_grd = df['GRD_SC'].values
            o_well_grd = df_alt['GRD_SC'].values

            df_core_window = df[df['MD'].isin(c_well_md)][df.columns]
            df_ofst_window = df_alt[df_alt['MD'].isin(o_well_md)][df_alt.columns]

            core_well_nm = core_well_names[i].split('.')[0].split('/')[-1]
            ofst_well_nm = offset_well_names_alt[j].split('.')[0].split('/')[-1]
            if (len(df_core_window) < len(df_ofst_window)):
              similarity(df_core_window, df_ofst_window, core_well_nm, ofst_well_nm,'md')
            else:
              similarity(df_ofst_window, df_core_window, core_well_nm, ofst_well_nm,'md')

            tvd_flag = True

           #

          else:


            lst_val_ss = srs_core_md.iloc[-1]  # last val in the MD column

            if lst_val_ss in df_alt_md.values:
                fst_val_owell = df_alt_md.iloc[0]
                # segment off the core well
                fst_idx_ss = srs_core_md[srs_core_md == fst_val_owell].index.values[0]
                lst_idx_ss = srs_core_md[srs_core_md == lst_val_ss].index.values[0]

                srs_core_md = srs_core_md[fst_idx_ss:(lst_idx_ss+1)]

                # now segment off the offset well

                fst_idx_owell = df_alt_md[df_alt_md == fst_val_owell].index.values[0]
                lst_idx_owell = df_alt_md[df_alt_md == lst_val_ss].index.values[0]

                owell_md = df_alt_md[fst_idx_owell:(lst_idx_owell+1)]


                # Do we really need this section ?  (NO)
                # now we have to add 50 elements at the beginning and at the end



                if fst_idx_owell >= 50:                   # check if the start index of the offset well > 50

                  nw_ofst_strt_idx =  fst_idx_owell - 50
                  pre_ofst_window = df_alt_md[nw_ofst_strt_idx:fst_idx_owell]
                  nw_ofst_window = pd.concat([pre_ofst_window,owell_md])
                  nw_ofst_window.index = np.arange(0,len(nw_ofst_window),1)
                else:

                  pre_ofst_window = df_alt_md[:fst_idx_owell]
                  nw_ofst_window = pd.concat([pre_ofst_window,owell_md])
                  nw_ofst_window.index = np.arange(0,len(nw_ofst_window),1)


                diff = df_alt_md.index.stop - lst_idx_owell

                if diff >= 50:
                  post_ofst_window = df_alt_md.iloc[lst_idx_owell+1 : lst_idx_owell+1+50]
                  nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])

                else:
                  post_ofst_window = df_alt_md[lst_idx_owell+1:]
                  nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                  nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series


                c_well_md = srs_core_md.values
                o_well_md = nw_ofst_window.values
                c_well_grd = df['GRD_SC'].values
                o_well_grd = df_alt['GRD_SC'].values

                # slice the segment for the GR values
                # df_core_window = df[df['MD'].isin(c_well_md)][['GRD_SC','MD']]
                # df_ofst_window = df_alt[df_alt['MD'].isin(o_well_md)][['GRD_SC','MD']]

                df_core_window = df[df['MD'].isin(c_well_md)][df.columns]
                df_ofst_window = df_alt[df_alt['MD'].isin(o_well_md)][df_alt.columns]

                core_well_nm = core_well_names[i].split('.')[0].split('/')[-1]
                ofst_well_nm = offset_well_names_alt[j].split('.')[0].split('/')[-1]
                if (len(df_core_window) < len(df_ofst_window)):
                  similarity(df_core_window, df_ofst_window, core_well_nm, ofst_well_nm,'md')
                else:
                  similarity(df_ofst_window, df_core_window, core_well_nm, ofst_well_nm,'md')

                tvd_flag = True


            if tvd_flag == False:
              tvd_un_val = srs_core_tvd.unique()  # ndarrray  (6,)
              df_tvd_mean = df.groupby(['TVD'])['GRD'].aggregate('mean').reset_index()
              df_tvd_mean_sc = df.groupby(['TVD'])['GRD_SC'].aggregate('mean').reset_index()
              elements = ','.join(str(element) for element in df_tvd_mean_sc['TVD'].values.tolist())
              shape = str(np.shape(df_tvd_mean_sc['TVD'].values)[0])
              fst_val_tvd_ss = df_tvd_mean_sc['TVD'].values[0]
              last_val_tvd_ss = df_tvd_mean_sc['TVD'].values[-1]
              if fst_val_tvd_ss in df_alt_md and last_val_tvd_ss in df_alt_md:
                strt_idx = df_alt_md[df_alt_md==fst_val_tvd_ss].index.values
                end_idx = df_alt_md[df_alt_md==last_val_tvd_ss].index.values
                o_well_tvd = df_alt_md[strt_idx[0]:(end_idx[0]+1)]
                new_ofst_window = df_alt[df_alt['MD'].isin(o_well_tvd)][df_alt.columns]

                # so we have to add 50 elements or lesser to the offset well over here to the newly formed series.
                if strt_idx[0] >= 50:
                  nw_ofst_strt_idx =  strt_idx[0] - 50
                  pre_ofst_window = df_alt_md[nw_ofst_strt_idx:strt_idx[0]]
                  nw_ofst_window = pd.concat([pre_ofst_window, new_ofst_window['MD']])
                  nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)

                else:
                  pre_ofst_window = df_alt_md[:strt_idx[0]]               #   all elements form the beginning
                  nw_ofst_window = pd.concat([pre_ofst_window, new_ofst_window['MD']])
                  nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series
                ow_end_idx = df_alt_md[df_alt_md==df_tvd_mean_sc['TVD'].iloc[-1]].index.values[0]   # return the last element of that list; step (a)

                diff =  df_alt_md.index.stop - ow_end_idx
                if diff >=50:
                  post_ofst_window = df_alt_md[ow_end_idx+1:ow_end_idx+1+50]
                  nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                  nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series

                else:
                  post_ofst_window = df_alt_md[ow_end_idx+1:]
                  nw_ofst_window = pd.concat([nw_ofst_window, post_ofst_window])
                  nw_ofst_window.index = np.arange(0, len(nw_ofst_window), 1)      # pandas series


                c_well_md = df_tvd_mean_sc['TVD'].values
                o_well_md = nw_ofst_window.values       # we have to take MD in this case
                df_tvd_mean_sc = pd.concat([df_tvd_mean_sc,df_tvd_mean['GRD']],axis=1)
                df_ofst_window = df_alt[df_alt['MD'].isin(o_well_md)][df_alt.columns]
                df_core_window = df_tvd_mean_sc
                core_well_nm = core_well_names[i].split('.')[0].split('/')[-1]
                ofst_well_nm = offset_well_names_alt[j].split('.')[0].split('/')[-1]
                if (len(df_core_window) < len(df_ofst_window)):
                  similarity(df_core_window, df_ofst_window, core_well_nm, ofst_well_nm,'tvd')
                else:
                  similarity(df_ofst_window, df_core_window, core_well_nm, ofst_well_nm,'tvd')







>>  ['/content/drive/My Drive/SSES/needed/alt_Emma_Will.csv', '/content/drive/My Drive/SSES/needed/alt_NC_Terry.csv', '/content/drive/My Drive/SSES/needed/alt_Smith_Brny.csv', '/content/drive/My Drive/SSES/needed/alt_Wolf_GE.csv']
143
case a
case a
case a
case a
